In [65]:
import numpy as np
import binascii

In [126]:
## Hepler functions for interconversion of bits and strings

def text_to_bits(text, encoding='utf-8', errors='surrogatepass'):
    bits = bin(int(binascii.hexlify(text.encode(encoding, errors)), 16))[2:]
    return bits.zfill(8 * ((len(bits) + 7) // 8))

def text_from_bits(bits, encoding='utf-8', errors='surrogatepass'):
    n = int(bits, 2)
    return int2bytes(n).decode(encoding, errors)

def int2bytes(i):
    hex_string = '%x' % i
    n = len(hex_string)
    return binascii.unhexlify(hex_string.zfill(n + (n & 1)))

In [3]:
## Main building blocks of Grain Cipher, namely a Linear Feedback Shift Register (LFSR)
## and a Non-linear Feedback Shift Register (NFSR)

lfsr = np.zeros(80,dtype=bool)
nfsr = np.zeros(80,dtype=bool)

In [4]:
## Initialise LFSR and NFSR using IV and Secret Key
## First load the NFSR with the key bits, bi = ki, 0 ≤ i ≤ 79,
## then load the first 64 bits of the LFSR with the IV, si = IVi, 0 ≤ i ≤ 63.
## The remaining bits of the LFSR are filled with ones, si = 1, 64 ≤ i ≤ 79.
## Because of this the LFSR cannot be initialized to the all zero state.

def init(iv,key):
    iv_bin = string2bits(iv)
    iv_bin = ''.join(iv_bin)
    lfsr[:64] = [bool(int(iv_bin[ix])) for ix in range(len(iv_bin))]
    lfsr[64:] = 1
    key_bin = string2bits(key)
    key_bin = ''.join(key_bin)
    nfsr[:] = [bool(int(key_bin[ix])) for ix in range(len(key_bin))]

In [5]:
init('absolute','california')

In [17]:
## The cipher is clocked 160 times without producing any running key
## The output of the filter function, h(x), is fed back and xored with the input, both to the LFSR and to the NFSR

def clock():
    hx=0
    fx=0
    gx=0
    global lfsr
    global nfsr
    for ix in range(160):
        fx = lfsr[62] ^ lfsr[51] ^ lfsr[38] ^ lfsr[23] ^ lfsr[13] ^ lfsr[0] ^ hx
        gx = hx ^ nfsr[0] ^ nfsr[63] ^ nfsr[60] ^ nfsr[52] ^ nfsr[45] ^ nfsr[37] ^ nfsr[33] ^ nfsr[28] ^ nfsr[21] ^ nfsr[15] ^ nfsr[19] ^ nfsr[0] ^ nfsr[63] & nfsr[60] ^ nfsr[37] & nfsr[33] ^ nfsr[15] & nfsr[9] ^ nfsr[60] & nfsr[52] & nfsr[45] ^ nfsr[33] & nfsr[28] & nfsr[21] ^ nfsr[63] & nfsr[45] & nfsr[28] & nfsr[9] ^ nfsr[60] & nfsr[52] & nfsr[37] & nfsr[33] ^ nfsr[63] & nfsr[60] & nfsr[21] & nfsr[15] ^ nfsr[63] & nfsr[60] & nfsr[52] & nfsr[45] & nfsr[37] ^ nfsr[33] & nfsr[28] & nfsr[21] & nfsr[15] & nfsr[9] ^ nfsr[52] & nfsr[45] & nfsr[37] & nfsr[33] & nfsr[28] & nfsr[21]
        x0 = lfsr[0]
        x1 = lfsr[25]
        x2 = lfsr[46]
        x3 = lfsr[64]
        x4 = nfsr[63]
        hx = x1 ^ x4 ^ x0 & x3 ^ x2 & x3 ^ x3 & x3 ^ x0 & x1 & x2 ^ x0 & x2 & x3 ^ x0 & x2 & x4 ^ x1 & x2 & x4 ^ x2 & x3 & x4
        lfsr[:-1] = lfsr[1:]
        lfsr[-1] = fx
        nfsr[:-1] = nfsr[1:]
        nfsr[-1] = gx

In [9]:
init_clock()

In [12]:
## Return a stream generator which implements the filter function

def gen_key_stream():
    hx = 0
    while True:
        fx = lfsr[62] ^ lfsr[51] ^ lfsr[38] ^ lfsr[23] ^ lfsr[13] ^ lfsr[0]
        gx = nfsr[0] ^ nfsr[63] ^ nfsr[60] ^ nfsr[52] ^ nfsr[45] ^ nfsr[37] ^ nfsr[33] ^ nfsr[28] ^ nfsr[21] ^ nfsr[15] ^ nfsr[19] ^ nfsr[0] ^ nfsr[63] & nfsr[60] ^ nfsr[37] & nfsr[33] ^ nfsr[15] & nfsr[9] ^ nfsr[60] & nfsr[52] & nfsr[45] ^ nfsr[33] & nfsr[28] & nfsr[21] ^ nfsr[63] & nfsr[45] & nfsr[28] & nfsr[9] ^ nfsr[60] & nfsr[52] & nfsr[37] & nfsr[33] ^ nfsr[63] & nfsr[60] & nfsr[21] & nfsr[15] ^ nfsr[63] & nfsr[60] & nfsr[52] & nfsr[45] & nfsr[37] ^ nfsr[33] & nfsr[28] & nfsr[21] & nfsr[15] & nfsr[9] ^ nfsr[52] & nfsr[45] & nfsr[37] & nfsr[33] & nfsr[28] & nfsr[21]
        x0 = lfsr[0]
        x1 = lfsr[25]
        x2 = lfsr[46]
        x3 = lfsr[64]
        x4 = nfsr[63]
        hx = x1 ^ x4 ^ x0 & x3 ^ x2 & x3 ^ x3 & x3 ^ x0 & x1 & x2 ^ x0 & x2 & x3 ^ x0 & x2 & x4 ^ x1 & x2 & x4 ^ x2 & x3 & x4
        lfsr[:-1] = lfsr[1:]
        lfsr[-1] = fx
        nfsr[:-1] = nfsr[1:]
        nfsr[-1] = gx
        yield hx

In [128]:
def encrypt(iv,key,plain):
    init(iv,key)
    clock()
    plain = text_to_bits(plain)
    stream = gen_key_stream()
    cipher = [str(int(bool(int(plain[ix]))^next(stream))) for ix in range(len(plain))]
    cipher = ''.join(cipher)
    return cipher

In [129]:
cipher = encrypt('absolute','california','First Test')

In [130]:
print cipher

10100111011001110111001000010001001010110000100001100101001000111010101101000111


In [137]:
def decrypt(iv,key,cipher):
    init(iv,key)
    clock()
    stream = gen_key_stream()
    plain = [str(int(bool(int(cipher[ix]))^next(stream))) for ix in range(len(cipher))]
    plain = ''.join(plain)
    plain = text_from_bits(plain)
    return plain

In [138]:
plain = decrypt('absolute','california',cipher)

In [139]:
print plain

First Test
